In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
from scipy import stats
import base_functions as bf
import cluster_base_functions as cbf

from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn import metrics

import warnings
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
# plt.rcParams["font.family"] = "Arial"
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
%matplotlib inline

In [ ]:
D = pickle.load(open('./median_data/combat_cluster_multi_modality_raw_data.pkl', 'rb'))
struc_feat = D['struc_feat']
conn_feat = D['conn_feat']
act_feat = D['act_feat']

basic_DF = D['basic_DF']
subjects = D['subjects']

struc_feat = stats.zscore(struc_feat, axis=0)
act_feat = stats.zscore(act_feat, axis=0)
conn_feat = stats.zscore(conn_feat, axis=0)

In [ ]:
from mlni.hydra_clustering import clustering

use_feat = 'anat' # which feature modality is used for clustering, should be 'anat', 'act', or 'conn'

dx_labels = bf.get_subject_info(basic_DF, subjects, ['diagnosis_group'])

if use_feat=='anat':
    hydra_feat = struc_feat.copy()
elif use_feat=='act':
    hydra_feat = act_feat.copy()
elif use_feat=='conn':
    hydra_feat = conn_feat.copy()

    
for care_label in [1,0]:# foucs on clustering patients or healthy

    labels = np.zeros_like(dx_labels)
    if care_label == 0:
        labels[dx_labels==0] = 1#be careful here about the labels
        labels[dx_labels==1] = -1
        output_dir = f'./MOD_HYDRA_{use_feat}_HC'
    elif care_label == 1:
        labels[dx_labels==0] = -1#be careful here about the labels
        labels[dx_labels==1] = 1
        output_dir = f'./MOD_HYDRA_{use_feat}_PT'

    session = ['ses-feat']*len(labels)
    df = pd.DataFrame({'participant_id':subjects, 'session_id':session, 'diagnosis':labels})

    for ifeat in range(hydra_feat.shape[1]):
        df[f'ROI{ifeat+1}'] = hydra_feat[:,ifeat]

    


    os.makedirs(output_dir, exist_ok=True)
    feature_tsv = f'{output_dir}/{use_feat}_features.tsv'
    df.to_csv(feature_tsv, index=False, sep='\t')


    k_min=2
    k_max=5


    cv_repetition=100
    clustering(feature_tsv, output_dir, k_min, k_max, cv_repetition, n_threads=5, save_models=False, cv_strategy='hold_out')
